In [1]:
from outlaw.nb_util import setup_nb
setup_nb()

In [2]:
import outlaw.inla as inla
import outlaw.quad as quad
import outlaw.berry as berry
import numpy as np
from numpy import nan
import jax
import jax.numpy as jnp

/Users/tbent/.mambaforge/envs/confirm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
def dot(DD, x, y):
    # return jnp.dot(x, y)
    return DD[jnp.dot(x, y)]
X = dot({1:10, 2:15, 3:20}, jnp.array([0,0,0,1]), jnp.array([1,2,3,4]))

TypeError: unhashable type: 'DeviceArray'

In [10]:
import numpyro.infer
import numpyro.distributions as dist

In [11]:
data = berry.figure2_data(N=10)

In [16]:
import scipy.special
logit_p1 = scipy.special.logit(0.3)
def model(data):
    mu = numpyro.sample("mu", dist.Normal(-1.34, 10))
    sigma2 = numpyro.sample("sigma2", dist.InverseGamma(0.0005, 0.000005))
    with numpyro.plate("j", 4):
        theta = numpyro.sample(
            "theta",
            dist.Normal(mu, jax.numpy.sqrt(sigma2)),
        )
        numpyro.sample(
            "y",
            dist.BinomialLogits(theta + logit_p1, total_count=data[...,1]),
            obs=data[...,0],
        )

In [23]:
# nuts_kwargs = dict(step_size=0.001, adapt_step_size=False)
nuts_kwargs = dict()
nuts_kernel = numpyro.infer.NUTS(model, **nuts_kwargs)
mcmc = numpyro.infer.MCMC(
    nuts_kernel,
    progress_bar=False,
    num_warmup=1000,
    num_samples=10000,
    jit_model_args=True
)
rng_key = jax.random.PRNGKey(0)

In [30]:
%%time
mcmc.run(rng_key, data[0])

CPU times: user 89.4 ms, sys: 1.44 ms, total: 90.9 ms
Wall time: 90.1 ms
